Assignment 1: Predict diabetes using Perceptron
Student: Pujan Maharjan (a1863495)
Course: Deep Learning Fundamentals

In [43]:
import torch
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
# !pip install ipywidgets


In [83]:

class Perceptron():
    def __init__(self, file_path, weights, loss_function_name, learning_rate, epoch) -> None:
        self.file_path = file_path
        self.weights = weights
        self.loss_function_name = loss_function_name
        self.learning_rate = learning_rate
        self.epoch = epoch

    def get_features_labels_from_file_data(self):
        X, y = load_svmlight_file(self.file_path)
        # convert X from scipy.sparce.csr.csr_matrix to numpy array
        X = X.toarray()
        # reshape y from (768,) to (768,1)
        y = y.reshape(-1, 1)
        return X,y

    def predict(self, X):
        return np.sign(np.dot(X, self.weights))

    def zero_one_loss(self, X, y):
        predictions = self.predict(X)
        losses = []
        # if correct prediction, then loss = 0, else loss = 1
        for i in range(len(y)):
            if y[i] == predictions[i]:
                losses.append(0)
            else:
                losses.append(1)

        return np.array(losses).reshape(-1,1)

    def train(self, X_train, y_train, X_val, y_val):
        train_data = []
        for epoch_number in range(self.epoch):
            # print('Epoch number: ', epoch_number)
            train_loss = None
            validation_loss = None
            train_accuracy = None
            if (self.loss_function_name == "zero_one_loss"):
                train_loss = self.zero_one_loss(X_train, y_train)
                validation_loss = self.zero_one_loss(X_val, y_val)
                
                # xl = X_train * train_loss               
                # yxl = y_train * xl                
                yxlr = self.learning_rate * y_train * X_train * train_loss           
                yxlr_sum = np.sum(yxlr, axis=0).reshape(-1,1)                
                self.weights = self.weights + yxlr_sum                
                train_accuracy = self.accuracy(X_train, y_train)
                
            validation_accuracy = self.accuracy(X_val, y_val)

            train_data.append({
                'learning_rate': self.learning_rate, 
                'epoch': epoch_number, 
                'train_loss': np.sum(train_loss), 
                'validation_loss': np.sum(validation_loss),
                'validation_accuracy': validation_accuracy,
                'train_accuracy': train_accuracy})

        # print('Training Completed')
        return train_data

    def accuracy(self, X_accuracy, y_accuracy):
        predictions_for_accuracy = self.predict(X_accuracy)
        accuracy_score_from_sk_learn = accuracy_score(y_accuracy, predictions_for_accuracy)
        return accuracy_score_from_sk_learn

    def split_train_validation_test(self, X, y):
        # reference to split (train/validation/test):
        #  https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
        # 0.25 * 0.8 = 0.2
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

        return X_train, X_val, X_test, y_train, y_val, y_test


In [84]:
# Experiments
# 1. Different Files (Without scaling vs scaled)
np.random.seed(0)
weights = np.random.uniform(low = -1, high = 1, size=8).reshape(-1,1)
for file_path in ['diabetes_scale.txt','diabetes.txt']:
    perceptron = Perceptron(
        file_path=file_path,
        weights=weights, 
        loss_function_name="zero_one_loss", 
        learning_rate=0.01, 
        epoch=10)
    X, y = perceptron.get_features_labels_from_file_data()
    X_train, X_val, X_test, y_train, y_val, y_test = perceptron.split_train_validation_test(X, y)
    train_data = perceptron.train(X_train, y_train, X_val, y_val)
    test_accuracy = perceptron.accuracy(X_test, y_test)
    print('File = ', file_path, ' test_accuracy = ', test_accuracy)
    # train_data_pd = pd.DataFrame(train_data)
    # print(train_data_pd)

File =  diabetes_scale.txt  test_accuracy =  0.6948051948051948
File =  diabetes.txt  test_accuracy =  0.6428571428571429


In [85]:
# 2. Learning Rate
np.random.seed(0)
weights = np.random.uniform(low = -1, high = 1, size=8).reshape(-1,1)
for learning_rate in [0.1,0.01,0.02,0.028,0.029,0.03,0.033,0.035,0.037,0.04,0.05,0.06,0.001,0.0001]:
    perceptron = Perceptron(
        file_path='diabetes_scale.txt',
        weights=weights, 
        loss_function_name="zero_one_loss", 
        learning_rate=learning_rate, 
        epoch=10)
    X, y = perceptron.get_features_labels_from_file_data()
    X_train, X_val, X_test, y_train, y_val, y_test = perceptron.split_train_validation_test(X, y)
    train_data = perceptron.train(X_train, y_train, X_val, y_val)
    test_accuracy = perceptron.accuracy(X_test, y_test)
    print('Learning Rate = ', learning_rate, ' test_accuracy = ', test_accuracy)

Learning Rate =  0.1  test_accuracy =  0.487012987012987
Learning Rate =  0.01  test_accuracy =  0.6948051948051948
Learning Rate =  0.02  test_accuracy =  0.6948051948051948
Learning Rate =  0.028  test_accuracy =  0.7142857142857143
Learning Rate =  0.029  test_accuracy =  0.7337662337662337
Learning Rate =  0.03  test_accuracy =  0.7077922077922078
Learning Rate =  0.033  test_accuracy =  0.6883116883116883
Learning Rate =  0.035  test_accuracy =  0.487012987012987
Learning Rate =  0.037  test_accuracy =  0.461038961038961
Learning Rate =  0.04  test_accuracy =  0.4675324675324675
Learning Rate =  0.05  test_accuracy =  0.474025974025974
Learning Rate =  0.06  test_accuracy =  0.474025974025974
Learning Rate =  0.001  test_accuracy =  0.44805194805194803
Learning Rate =  0.0001  test_accuracy =  0.33116883116883117


In [86]:
# 3. Epoch
np.random.seed(0)
weights = np.random.uniform(low = -1, high = 1, size=8).reshape(-1,1)
for epoch in range(10, 100, 10):
    perceptron = Perceptron(
        file_path='diabetes_scale.txt',
        weights=weights, 
        loss_function_name="zero_one_loss", 
        learning_rate=0.029, 
        epoch=epoch)
    X, y = perceptron.get_features_labels_from_file_data()
    X_train, X_val, X_test, y_train, y_val, y_test = perceptron.split_train_validation_test(X, y)
    train_data = perceptron.train(X_train, y_train, X_val, y_val)
    test_accuracy = perceptron.accuracy(X_test, y_test)
    print('Epoch = ', epoch, ' test_accuracy = ', test_accuracy)

Epoch =  10  test_accuracy =  0.7337662337662337
Epoch =  20  test_accuracy =  0.6948051948051948
Epoch =  30  test_accuracy =  0.7662337662337663
Epoch =  40  test_accuracy =  0.6688311688311688
Epoch =  50  test_accuracy =  0.6688311688311688
Epoch =  60  test_accuracy =  0.7922077922077922
Epoch =  70  test_accuracy =  0.7662337662337663
Epoch =  80  test_accuracy =  0.6688311688311688
Epoch =  90  test_accuracy =  0.7272727272727273


In [87]:
# 4. Weights
# low,high pair
weights_pairs = [(-1,1), (0,1), (-2,2),(-0.5,0.5)]
np.random.seed(0)
for weight_pair in weights_pairs:
    low, high = weight_pair
    weights = np.random.uniform(low=low, high = high, size=8).reshape(-1,1)
    perceptron = Perceptron(
        file_path='diabetes_scale.txt',
        weights=weights, 
        loss_function_name="zero_one_loss", 
        learning_rate=0.029, 
        epoch=60)
    X, y = perceptron.get_features_labels_from_file_data()
    X_train, X_val, X_test, y_train, y_val, y_test = perceptron.split_train_validation_test(X, y)
    train_data = perceptron.train(X_train, y_train, X_val, y_val)
    test_accuracy = perceptron.accuracy(X_test, y_test)
    print('Weight = ', weight_pair, ' test_accuracy = ', test_accuracy)

Weight =  (-1, 1)  test_accuracy =  0.7922077922077922
Weight =  (0, 1)  test_accuracy =  0.6688311688311688
Weight =  (-2, 2)  test_accuracy =  0.7857142857142857
Weight =  (-0.5, 0.5)  test_accuracy =  0.7727272727272727
